In [ ]:
!pip install -q streamlit
!pip install openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%writefile app.py
import streamlit as st
from openai import OpenAI
import time

client = OpenAI(
    api_key=""
)

PDF_DIR = "/content/drive/My Drive/Projects/Project records/Nona works"
fair_pdf_path = f"{PDF_DIR}/fair_publication.pdf"
miriam_pdf_path = f"{PDF_DIR}/miriam_publication.pdf"

# @st.cache_resource
# def initialize_assistant():
#     with open(fair_pdf_path, 'rb') as pdf_file:
#         fair_file = client.files.create(file=pdf_file, purpose='assistants')

#     with open(miriam_pdf_path, 'rb') as pdf_file:
#         miriam_file = client.files.create(file=pdf_file, purpose='assistants')

#     assistant = client.beta.assistants.create(
#         name="FAIR & MIRIAM Research Assistant",
#         instructions="You are an expert on the FAIR principles and MIRIAM publications.",
#         model="gpt-3.5-turbo",
#         tools=[{"type": "code_interpreter"}],
#         tool_resources={"code_interpreter": {"file_ids": [fair_file.id, miriam_file.id]}}
#     )
#     return assistant

def main():
    st.title("FAIR & MIRIAM Assistant")
    # assistant = initialize_assistant()
    assistant_id = 'asst_H37be609KUxWRUur5uMu0WyC'

    if "thread_id" not in st.session_state:
        thread = client.beta.threads.create()
        st.session_state.thread_id = thread.id

    user_input = st.text_area("Ask a question:", height=100)

    if st.button("Get Response"):
        message = client.beta.threads.messages.create(
            thread_id=st.session_state.thread_id,
            role="user",
            content=user_input
        )

        run = client.beta.threads.runs.create(
            thread_id=st.session_state.thread_id,
            assistant_id=assistant_id
        )

        while run.status in ["queued", "in_progress"]:
            run = client.beta.threads.runs.retrieve(
                thread_id=st.session_state.thread_id,
                run_id=run.id
            )
            time.sleep(0.5)

        messages = client.beta.threads.messages.list(thread_id=st.session_state.thread_id)
        st.text_area("Response:", value=messages.data[0].content[0].text.value, height=300, disabled=True)

if __name__ == "__main__":
    main()

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501